In [34]:
#Text preprocessing. Cleaning/ preparing data.
#Tokenization. Dividing sentences into words.
#Stopword removal. Removing irrelevant words.
#Stemming & Lemmatization. Reducing words to base form; Grammar
#Named Entry recognition. Extract names , places . dates
#Text Vectorisation. # Convert text to Numbers.
# modelling. Create your model.

In [35]:
# Import necessary libraries.
import pandas as pd
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report



In [36]:
data = pd.read_csv('Twitter_Data.csv')

In [37]:
display(data.head())

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [38]:
#Step 1; Text Preprocessing.
#Drop missing rows.
data.dropna(inplace=True)

# Convert texts to lowercase.
type(data['clean_text'])
data['clean_text']= data['clean_text'].astype(str)
data['clean_text'] = data['clean_text'].str.lower()

#Remove symbols.
def clean_text(text):
   text = re.sub(r'http\S+|www.\S+', '', text) #Remove Urls.
   text = re.sub(r'@\w+|#\w+', '', text) #Remove mentions and hashtags.
   text = re.sub(r'\d+', '', text) #Remove digits.
   text = re.sub(r'[^\w\s]', '', text) #Remove punctuation.
   return text

data['clean_text'] = data['clean_text'].apply(clean_text)
data.head()

,clean_text,category
0,when modi promised minimum government maximum ...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [39]:
# Step 2; Tokenization
#Download tokenizer model
import nltk
nltk.download('punkt')

# Create to process clean_text
class TextProcessing:
  def __init__(self,column):
    self.column = column

  def tokenizeText(self, text):
    return word_tokenize(str(text))

  def apply(self, df):
    df['tokens'] = df[self.column].map(self.tokenizeText)
    return df

# Apply tokenization on column 'clean_text'
processor = TextProcessing(column='clean_text')
data = processor.apply(data)

data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,clean_text,category,tokens
0,when modi promised minimum government maximum ...,-1.0,"[when, modi, promised, minimum, government, ma..."
1,talk all the nonsense and continue all the dra...,0.0,"[talk, all, the, nonsense, and, continue, all,..."
2,what did just say vote for modi welcome bjp t...,1.0,"[what, did, just, say, vote, for, modi, welcom..."
3,asking his supporters prefix chowkidar their n...,1.0,"[asking, his, supporters, prefix, chowkidar, t..."
4,answer who among these the most powerful world...,1.0,"[answer, who, among, these, the, most, powerfu..."


In [40]:
display(data.head())

,clean_text,category,tokens
0,when modi promised minimum government maximum ...,-1.0,"[when, modi, promised, minimum, government, ma..."
1,talk all the nonsense and continue all the dra...,0.0,"[talk, all, the, nonsense, and, continue, all,..."
2,what did just say vote for modi welcome bjp t...,1.0,"[what, did, just, say, vote, for, modi, welcom..."
3,asking his supporters prefix chowkidar their n...,1.0,"[asking, his, supporters, prefix, chowkidar, t..."
4,answer who among these the most powerful world...,1.0,"[answer, who, among, these, the, most, powerfu..."


In [41]:
# Step 3: Removing Stopwords
# Downloading stopwords
nltk.download('stopwords')

#defining english stopwords
english_stopwords = set(stopwords.words('english'))

#Function to remove the stopwords
def remove_stopwords(row_tokens):
    cleaned = []
    for word in row_tokens:
        if word not in english_stopwords:
            cleaned.append(word)
    return cleaned

# applying the function to the data
data['stopword_tokens'] = data['tokens'].apply(remove_stopwords)
data['tokens']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,tokens
0,"[when, modi, promised, minimum, government, ma..."
1,"[talk, all, the, nonsense, and, continue, all,..."
2,"[what, did, just, say, vote, for, modi, welcom..."
3,"[asking, his, supporters, prefix, chowkidar, t..."
4,"[answer, who, among, these, the, most, powerfu..."
...,...
162975,"[why, these, crores, paid, neerav, modi, not, ..."
162976,"[dear, rss, terrorist, payal, gawar, what, abo..."
162977,"[did, you, cover, her, interaction, forum, whe..."
162978,"[there, big, project, came, into, india, modi,..."


In [42]:
data.head()

,clean_text,category,tokens,stopword_tokens
0,when modi promised minimum government maximum ...,-1.0,"[when, modi, promised, minimum, government, ma...","[modi, promised, minimum, government, maximum,..."
1,talk all the nonsense and continue all the dra...,0.0,"[talk, all, the, nonsense, and, continue, all,...","[talk, nonsense, continue, drama, vote, modi]"
2,what did just say vote for modi welcome bjp t...,1.0,"[what, did, just, say, vote, for, modi, welcom...","[say, vote, modi, welcome, bjp, told, rahul, m..."
3,asking his supporters prefix chowkidar their n...,1.0,"[asking, his, supporters, prefix, chowkidar, t...","[asking, supporters, prefix, chowkidar, names,..."
4,answer who among these the most powerful world...,1.0,"[answer, who, among, these, the, most, powerfu...","[answer, among, powerful, world, leader, today..."


In [43]:
#Step 4: Stemming.
from nltk.stem import PorterStemmer

# Defining the stemmer
stemmer = PorterStemmer()

# Function for stemming the text
def stem_tokens(tokens):
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens

data['stemmed_tokens'] = data['stopword_tokens'].apply(stem_tokens)

In [44]:
data.head()

,clean_text,category,tokens,stopword_tokens,stemmed_tokens
0,when modi promised minimum government maximum ...,-1.0,"[when, modi, promised, minimum, government, ma...","[modi, promised, minimum, government, maximum,...","[modi, promis, minimum, govern, maximum, gover..."
1,talk all the nonsense and continue all the dra...,0.0,"[talk, all, the, nonsense, and, continue, all,...","[talk, nonsense, continue, drama, vote, modi]","[talk, nonsens, continu, drama, vote, modi]"
2,what did just say vote for modi welcome bjp t...,1.0,"[what, did, just, say, vote, for, modi, welcom...","[say, vote, modi, welcome, bjp, told, rahul, m...","[say, vote, modi, welcom, bjp, told, rahul, ma..."
3,asking his supporters prefix chowkidar their n...,1.0,"[asking, his, supporters, prefix, chowkidar, t...","[asking, supporters, prefix, chowkidar, names,...","[ask, support, prefix, chowkidar, name, modi, ..."
4,answer who among these the most powerful world...,1.0,"[answer, who, among, these, the, most, powerfu...","[answer, among, powerful, world, leader, today...","[answer, among, power, world, leader, today, t..."


In [45]:
#Step 5: Lemmatizing texts
nltk.download('wordnet')

# Defining the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function forlemmatizing the text
def lemmatize_tokens(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmatized_tokens

data['lemmatized_tokens'] = data['stopword_tokens'].apply(lemmatize_tokens)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
data.head()

,clean_text,category,tokens,stopword_tokens,stemmed_tokens,lemmatized_tokens
0,when modi promised minimum government maximum ...,-1.0,"[when, modi, promised, minimum, government, ma...","[modi, promised, minimum, government, maximum,...","[modi, promis, minimum, govern, maximum, gover...","[modi, promised, minimum, government, maximum,..."
1,talk all the nonsense and continue all the dra...,0.0,"[talk, all, the, nonsense, and, continue, all,...","[talk, nonsense, continue, drama, vote, modi]","[talk, nonsens, continu, drama, vote, modi]","[talk, nonsense, continue, drama, vote, modi]"
2,what did just say vote for modi welcome bjp t...,1.0,"[what, did, just, say, vote, for, modi, welcom...","[say, vote, modi, welcome, bjp, told, rahul, m...","[say, vote, modi, welcom, bjp, told, rahul, ma...","[say, vote, modi, welcome, bjp, told, rahul, m..."
3,asking his supporters prefix chowkidar their n...,1.0,"[asking, his, supporters, prefix, chowkidar, t...","[asking, supporters, prefix, chowkidar, names,...","[ask, support, prefix, chowkidar, name, modi, ...","[asking, supporter, prefix, chowkidar, name, m..."
4,answer who among these the most powerful world...,1.0,"[answer, who, among, these, the, most, powerfu...","[answer, among, powerful, world, leader, today...","[answer, among, power, world, leader, today, t...","[answer, among, powerful, world, leader, today..."
